In [2]:
sc

StatementMeta(sparkpool, 2, 2, Finished, Available, Finished)

<SparkContext master=yarn appName=Notebook 1_sparkpool_1721299056>

In [3]:
spark

StatementMeta(sparkpool, 2, 3, Finished, Available, Finished)

StatementMeta(sparkpool, 2, 9, Finished, Available, Finished)

In [10]:
df_static.schema

StatementMeta(sparkpool, 2, 10, Finished, Available, Finished)

StructType([StructField('VendorID', IntegerType(), True), StructField('PickUpTime', TimestampType(), True), StructField('TripDistance', DoubleType(), True), StructField('DropTime', TimestampType(), True), StructField('PickUpLocationID', IntegerType(), True), StructField('DropLocationID', IntegerType(), True), StructField('fare_amount', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', IntegerType(), True), StructField('trip_type', IntegerType(), True), StructField('TripType', StringType(), True)])

In [11]:
fileStreamDF = spark.readStream.schema(df_static.schema)\
.parquet("abfss://employeefs@datalakenewdemo.dfs.core.windows.net/synapse/workspaces/azure-worksynapse/warehouse/green_trip_txitbl/*.parquet")

StatementMeta(sparkpool, 2, 11, Finished, Available, Finished)

In [12]:
fileStreamDF.select("*")

StatementMeta(sparkpool, 2, 12, Finished, Available, Finished)

DataFrame[VendorID: int, PickUpTime: timestamp, TripDistance: double, DropTime: timestamp, PickUpLocationID: int, DropLocationID: int, fare_amount: double, total_amount: double, payment_type: int, trip_type: int, TripType: string]

In [15]:
print(fileStreamDF.columns)

StatementMeta(sparkpool, 2, 15, Finished, Available, Finished)

['VendorID', 'PickUpTime', 'TripDistance', 'DropTime', 'PickUpLocationID', 'DropLocationID', 'fare_amount', 'total_amount', 'payment_type', 'trip_type', 'TripType']


In [21]:
df_static = spark.read.format("parquet").option("path",
"abfss://employeefs@datalakenewdemo.dfs.core.windows.net/synapse/workspaces/azure-worksynapse/warehouse/green_trip_txitbl/*.parquet").load()

from pyspark.sql.functions import *

fileStreamDF1 = fileStreamDF.select('VendorID','PickUpTime','TripDistance','DropTime','PickUpLocationID',
'DropLocationID','fare_amount','total_amount','payment_type','trip_type','TripType').\
withColumnRenamed("fare_amount","FareAmount").\
withColumnRenamed("total_amount","TotalAmount")

query = fileStreamDF1.writeStream.outputMode("append")\
.format("json")\
.option("path","abfss://employeefs@datalakenewdemo.dfs.core.windows.net/output_data1")\
.option("checkpointlocation","abfss://employeefs@datalakenewdemo.dfs.core.windows.net/checkpoint1/")\
.start()
query.awaitTermination()

StatementMeta(sparkpool, 2, 21, Finished, Cancelled, Cancelled)

In [24]:
df_static.select('VendorID','TripType','payment_type').distinct().show()

StatementMeta(sparkpool, 2, 24, Finished, Available, Finished)

Py4JJavaError: An error occurred while calling o4284.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 4 times, most recent failure: Lost task 0.3 in stage 7.0 (TID 31) (vm-8ed61506 executor 1): org.apache.spark.SparkFileNotFoundException: Operation failed: "The specified path does not exist.", 404, HEAD, https://datalakenewdemo.dfs.core.windows.net/employeefs/synapse/workspaces/azure-worksynapse/warehouse/green_trip_txitbl/part-00002-7fbb5d89-a746-4074-adb4-90a35fa3da38-c000.snappy.parquet?upn=false&action=getStatus&timeout=90
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:794)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$

Create a Pyspark Structured Streaming to Show Total-Income Generated By Each VendorID,its TripType and Payment Type.

In [26]:
df_static = spark.read.format("parquet").option("path",
"abfss://employeefs@datalakenewdemo.dfs.core.windows.net/synapse/workspaces/azure-worksynapse/warehouse/green_trip_txitbl/*.parquet").load()

from pyspark.sql.functions import *

fileStreamDF1 = fileStreamDF.select('VendorID','PickUpTime','TripDistance','DropTime','PickUpLocationID',
'DropLocationID','fare_amount','total_amount','payment_type','trip_type','TripType').\
withColumnRenamed("fare_amount","FareAmount").\
withColumnRenamed("total_amount","TotalAmount")

total_df = fileStreamDF1.groupBy("VendorID","TripType","payment_type")\
.agg(sum("TotalAmount"))

query = fileStreamDF1.writeStream.outputMode("append")\
.format("csv")\
.option("path","abfss://employeefs@datalakenewdemo.dfs.core.windows.net/output_data2")\
.option("checkpointlocation","abfss://employeefs@datalakenewdemo.dfs.core.windows.net/checkpoint2/")\
.start()
query.awaitTermination()

StatementMeta(sparkpool, 2, 26, Finished, Cancelled, Cancelled)